In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
txt = "Data\Data 2.txt"
df = pd.read_csv(txt, sep="\t")

In [ ]:
df.set_index("type", inplace = True)
df.drop(index = ["type"], inplace = True)
df.reset_index(inplace = True)
# df.head(20)

In [ ]:
df.fillna(method = 'ffill', inplace = True)
df["eos"].fillna(1, inplace = True)
df["desc"].fillna(df['name'], inplace = True)
# df.head(10)

In [ ]:
for col in df.columns:
    if df[col].nunique() <= 2:
        df.drop(columns = [col], inplace = True)

df.drop(columns = ['color', 'name'], inplace = True)
# df.head(10)

In [ ]:
gdf = gpd.GeoDataFrame(data = df, crs = "EPSG:4326", geometry = gpd.points_from_xy(df.longitude, df.latitude)) # type: ignore
gdf.drop(columns = ["longitude", "latitude"], inplace = True)
gdf = gdf.to_crs("EPSG:32737")

gdf['fid'] = gdf.index + 1
fid = gdf.pop('fid')
gdf.insert(0, 'fid', fid)
gdf.head(10)

In [ ]:
e = 1618010.606367111
f = -1741978.211608887

a = 0.9916765364496314
d = 1.175648448821448

b = -0.1635616585599564
c = 0.02551828620198648

In [ ]:
# Transform to corresponding coordinates
gdf2 = gpd.GeoDataFrame((gdf.affine_transform([a, b, c, d, e, f])))
gdf2.geometry = gdf2[0]
gdf2.drop(columns = [0], inplace = True)
gdf2.set_crs("EPSG:32737")
gdf2.head(10)


In [ ]:
gdf.columns
cols = gdf.iloc[:, :3]
cols

data = pd.concat([cols, gdf2], axis = 1)
data = gpd.GeoDataFrame(data, crs = "EPSG:32737")
# data.head(10)

In [ ]:
data2 = data[['fid', 'desc', 'altitude (m)']]
data2['x'] = data.geometry.x
data2['y'] = data.geometry.y

names = {
    'fid': 'P',
    'y': 'N',
    'x': 'E',
    'altitude (m)': 'Z',
    'desc': 'D'
}

data2.rename(columns = names, inplace = True)
data2 = data2[['P', 'N', 'E', 'Z', 'D']]
data2.head(10)


In [ ]:
export = 'C:/Users/LUNA/Desktop/data.csv'
# data2.to_csv(export, index=False, header=False)
